In [1]:
import sparknlp
import pandas as pd
import os
import time
from helper import preprocess, preprocess_sparknlp
from pyspark.sql import SparkSession

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import os 
import pandas as pd
import json
import nltk
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import word_tokenize, sent_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import sparknlp
import pyspark
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml.feature import CountVectorizer
from pyspark.ml.feature import IDF
from pyspark.ml.clustering import LDA

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lam.nguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Evaluation

In [3]:
sparknlp_session = sparknlp.start(params={"spark.driver.host": "localhost",
                                          "spark.driver.port": "9999",
                                          "spark.driver.bindAddress": "127.0.0.1",
                                          "spark.driver.maxResultSize": "2g"})

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", sparknlp_session.version)

:: loading settings :: url = jar:file:/Users/lam.nguyen/Desktop/GithubClone/BigData_Final/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/lam.nguyen/.ivy2/cache
The jars for the packages stored in: /Users/lam.nguyen/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f678e84f-2e67-4752-89ac-72a6d09ff734;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;6.0.0 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sdk-core;1.12.500 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found joda-time#joda-time;2.8.1 in central
	found com.amazonaws#jmespath-java;1.12.500

Spark NLP version:  6.0.0
Apache Spark version:  3.5.5


In [4]:
# sparknlp_session.stop()

NTLK preprocessing vs SparkNLP preprocessing

In [4]:
corpus = pd.read_json(os.path.join("..", "data", "merged", "corpus.json"))

In [6]:
num_rows = [row for row in range(1000, 11000, 1000)]

In [28]:
def preprocess_sparknlp(
        data: pd.DataFrame,
        sparknlp_session: pyspark.sql.session.SparkSession
) -> pyspark.sql.dataframe.DataFrame:
    """Performs the preprocessing using SparkNLP
    
    Args:
        data: input file as pandas.DataFrame
        sparknlp_session: a SparkSession
    Returns:
        A pyspark.sql.dataframe.DataFrame which has vector form of the input
    """
    # Change from pandas.DataFrame to Spark DataFrame
    sparknlp_df = sparknlp_session.createDataFrame(data)
    sparknlp_df.cache()
    # sparknlp_df = sparknlp_df.repartition(10)
    # Document Assembler: Converts input text into a suitable format for NLP processing
    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")\
        .setCleanupMode("shrink")

    # Sentence tokenisation
    sentenceDetector = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentences')

    # Word tokenisation 
    tokeniser = Tokenizer()\
        .setInputCols(["sentences"]) \
        .setOutputCol("token")
    
    # Text cleaning and Normalisation
    normaliser = Normalizer()\
        .setInputCols("token")\
        .setOutputCol("normalised")\
        .setLowercase(True)\
        .setCleanupPatterns(["[^a-zA-Z\s]"])

    # Stopword Removal
    stopwords_cleaner = StopWordsCleaner()\
        .setInputCols("normalised")\
        .setOutputCol("stopwords_removed")\
        .setCaseSensitive(False)

    # Lemmatisation
    lemmatizer = LemmatizerModel.pretrained()\
        .setInputCols(["stopwords_removed"])\
        .setOutputCol("lemma")
    
    # Finisher 
    finisher = Finisher() \
        .setInputCols("lemma") \
        .setOutputCols("finish") \
        .setIncludeMetadata(False) # set to False to remove metadata
    
    # Pipeline
    pipeline = Pipeline().setStages([
            documentAssembler,
            sentenceDetector,
            tokeniser,
            normaliser,
            stopwords_cleaner,
            lemmatizer,
            finisher
            ])
    result = pipeline.fit(sparknlp_df).transform(sparknlp_df)    
    
    result.cache()
    
    # Word Embeddings (turns into vector)
    ## Term-Frequency (TF) transform
    tfizer = CountVectorizer(inputCol='finish', outputCol='tf_features')
    tf_model = tfizer.fit(result)
    tf_result = tf_model.transform(result)
    
    ## TF-IDF trasnform
    idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
    idf_model = idfizer.fit(tf_result)
    tfidf_result = idf_model.transform(tf_result)

    return result, tfidf_result

In [5]:
def pipeline_model(
        train_data: pd.DataFrame,
        spark_session: pyspark.sql.session.SparkSession
) -> pyspark.ml.pipeline.PipelineModel:
    """Trains a Pipeline model used to be applied for preprocessing phase

    Args:
        train_data: data as pandas.DataFrame
        spark_session: a SparkSession
    Returns:
        A trained PipelineModel
    """
    
    sparknlp_df = spark_session.createDataFrame(train_data)
    # Document Assembler: Converts input text into a suitable format for NLP processing
    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")\
        .setCleanupMode("shrink")

    # Sentence tokenisation
    sentenceDetector = SentenceDetector()\
        .setInputCols(['document'])\
        .setOutputCol('sentences')

    # Word tokenisation 
    tokeniser = Tokenizer()\
        .setInputCols(["sentences"]) \
        .setOutputCol("token")

    # Text cleaning and Normalisation
    normaliser = Normalizer()\
        .setInputCols("token")\
        .setOutputCol("normalised")\
        .setLowercase(True)\
        .setCleanupPatterns(["[^a-zA-Z\s]"])

    # Stopword Removal
    stopwords_cleaner = StopWordsCleaner()\
        .setInputCols("normalised")\
        .setOutputCol("stopwords_removed")\
        .setCaseSensitive(False)

    # Lemmatisation
    lemmatizer = LemmatizerModel.pretrained()\
        .setInputCols(["stopwords_removed"])\
        .setOutputCol("lemma")

    # Finisher 
    finisher = Finisher() \
        .setInputCols("lemma") \
        .setOutputCols("finish") \
        .setIncludeMetadata(False) # set to False to remove metadata

    # Pipeline
    pipeline = Pipeline().setStages([
            documentAssembler,
            sentenceDetector,
            tokeniser,
            normaliser,
            stopwords_cleaner,
            lemmatizer,
            finisher
            ])
    result = pipeline.fit(sparknlp_df)
    return result

25/05/03 23:28:13 WARN StopWordsCleaner: Default locale set was [en_VN]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ]

25/05/03 23:30:25 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/05/03 23:30:26 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ / ]Download done! Loading the resource.
[ — ]

[OK!]


In [15]:
def preprocess_sparknlp(
        data: pd.DataFrame,
        spark_session: pyspark.sql.session.SparkSession,
        pipeline_model: pyspark.ml.pipeline.PipelineModel
) -> pyspark.sql.dataframe.DataFrame:
    """Performs the preprocessing using a trained PipelineModel
    
    Args:
        data: input file as pandas.DataFrame
        sparknlp_session: a SparkSession
        pipeline_model: an instance of pyspark.ml.pipeline.PipelineModel to perform preprocessing
    Returns:
        A pyspark.sql.dataframe.DataFrame which has vector form of the input
    """
    # Change from pandas.DataFrame to Spark DataFrame
    sparknlp_df = spark_session.createDataFrame(data)
    result = pipeline_model.transform(sparknlp_df)    
    
    # Word Embeddings (turns into vector)
    ## Term-Frequency (TF) transform
    tfizer = CountVectorizer(inputCol='finish', outputCol='tf_features')
    tf_model = tfizer.fit(result)
    tf_result = tf_model.transform(result)
    
    ## TF-IDF trasnform
    idfizer = IDF(inputCol='tf_features', outputCol='tf_idf_features')
    idf_model = idfizer.fit(tf_result)
    tfidf_result = idf_model.transform(tf_result)

    return tfidf_result

pyspark.ml.pipeline.PipelineModel

In [41]:
nltk_time = []
sparknlp_time = []
for row in num_rows:
    subset_df = corpus.copy(deep=True).head(row)
    # NLTK time
    nltk_start = time.time()
    nlkt_result = preprocess(subset_df)
    nltk_end = time.time()
    nltk_time.append(nltk_end - nltk_start)

    # SparkNLP time
    sparknlp_start = time.time()
    sparknlp_result = preprocess_sparknlp(subset_df,
                                      sparknlp_session)
    sparknlp_end = time.time()
    sparknlp_time.append(sparknlp_end - sparknlp_start)

KeyboardInterrupt: 

In [16]:
nltk_time

hiih
0.0009012222290039062


In [ ]:
sparknlp_time

Throughput, time, speedup when use SparkNLP + PySpark to preprocess and run vs Scikit-learn to train model

Algorithm performance: Coherence (PMI) and Diversity (PUW)